In [1]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
print(cv.__version__)

4.2.0


In [2]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")
V = ry.ConfigurationViewer()
V.setConfiguration(RealWorld)

In [3]:
#change some colors
RealWorld.getFrame("obj0").setColor([1,0,0])
RealWorld.getFrame("obj1").setColor([1,0,0])
RealWorld.getFrame("obj2").setColor([1,1,0])
RealWorld.getFrame("obj3").setColor([1,0,1])
RealWorld.getFrame("obj4").setColor([0,1,1])

#you can also change the shape & size
RealWorld.getFrame("obj0").setShape(ry.ST.ssBox, [.1, .1, .1, .02])
RealWorld.getFrame("obj0").setPosition([0.2, .2, 2.])

#remove some objects
for o in range(1,30):
    name = "obj%i" % o
    print("deleting", name)
    RealWorld.delFrame(name)

V.recopyMeshes(RealWorld)
V.setConfiguration(RealWorld)

deleting obj1
deleting obj2
deleting obj3
deleting obj4
deleting obj5
deleting obj6
deleting obj7
deleting obj8
deleting obj9
deleting obj10
deleting obj11
deleting obj12
deleting obj13
deleting obj14
deleting obj15
deleting obj16
deleting obj17
deleting obj18
deleting obj19
deleting obj20
deleting obj21
deleting obj22
deleting obj23
deleting obj24
deleting obj25
deleting obj26
deleting obj27
deleting obj28
deleting obj29


In [4]:
#-- Add REAL WORLD configuration and camera
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

C = ry.Config()
C.addFile('../../scenarios/pandasTable.g')
#V = ry.ConfigurationViewer()

cameraFrame = C.frame("camera")


In [5]:
#S = ry.Simulation(C, ry.SimulatorEngine.physx, True)
V.setConfiguration(C)

In [6]:
#the focal length
f = 0.895
f = f * 360.
fxfypxpy = [f, f, 320., 180.]

In [7]:
def depth_data_to_point(pt, fxypxy):
    pt[0] =  pt[2] * (pt[0] - fxypxy[2]) / fxypxy[0]
    pt[1] = -pt[2] * (pt[1] - fxypxy[3]) / fxypxy[1]
    pt[2] = -pt[2]
    return pt
def camera_to_world(pt, cameraFrame, fxfypxpy):
    """
    pt = np.array([u, v, distance])
    """
    cam_rot = cameraFrame.getRotationMatrix()
    cam_trans = cameraFrame.getPosition()
    pt = depth_data_to_point(pt, fxfypxpy) @ cam_rot.T + cam_trans
    return pt

In [8]:
points = []
tau = .01
for t in range(300):
    time.sleep(0.01)
    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
        cameraFrame.setPointCloud(points, rgb)
        V.recopyMeshes(C)
        V.setConfiguration(C)
        
#         rgb = cv.cvtColor(rgb, cv.COLOR_BGR2RGB)
        hsv = cv.cvtColor(rgb, cv.COLOR_RGB2HSV)
        
        # Range for lower red
        lower_red = np.array([0,120,70])
        upper_red = np.array([10,255,255])
        mask1 = cv.inRange(hsv, lower_red, upper_red)
        # Range for upper range
        lower_red = np.array([170,120,70])
        upper_red = np.array([180,255,255])
        mask2 = cv.inRange(hsv,lower_red,upper_red)
        # Generating the final mask to detect red color
        mask = mask1+mask2
        res = cv.bitwise_and(rgb,rgb,mask = mask)
        contours, hier = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        if len(contours) != 0:
#             cv.drawContours(rgb, contours, -1, (0,255,0), 3)
#             cv.drawContours(mask, contours, -1, (0,255,0), 3)
            for c in contours:
#                 c = max(contours, key = cv.contourArea)
                x,y,w,h = cv.boundingRect(c)
                cv.rectangle(rgb,(x,y),(x+w,y+h),(0,255,0),2)
                x_r = x+int(w/2)
                y_r = y+int(h/2)
                cv.circle(rgb,(x_r, y_r), 3, (0,255,0), -1)
#                 print(x_r, y_r)
                pt = [y_r, x_r, depth[y_r, x_r]]
                depth_world = camera_to_world(pt,cameraFrame, fxfypxpy)
                
                font                   = cv.FONT_HERSHEY_SIMPLEX
                fontScale              = 0.5
                fontColor              = (255,255,255)
                lineType               = 2
                
#                 cv.putText(rgb,str(depth[y_r, x_r]), 
#                             (x_r,y_r),
#                             font, 
#                             fontScale,
#                             fontColor,
#                             lineType)
                cv.putText(rgb,str(depth_world), 
                            (x_r,y_r),
                            font, 
                            fontScale,
                            fontColor,
                            lineType)
        # Depth Points to World Coordniates
#         depth_world = np.empty([depth.shape[0],depth.shape[1],3])
#         for i in range(depth.shape[0]):
#             for j in range(depth.shape[1]):
#                 pt = [i,j,depth[i,j]]
#                 depth_world[i,j] = camera_to_world(pt,cameraFrame, fxfypxpy)
        
        if len(rgb)>0: cv.imshow('OPENCV - red Mask', cv.cvtColor(res, cv.COLOR_BGR2RGB))
        if len(rgb)>0: cv.imshow('OPENCV - rgb', cv.cvtColor(rgb, cv.COLOR_BGR2RGB))
        if len(depth)>0: cv.imshow('OPENCV - depth', 0.5* depth)
        
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        
    S.step([], tau, ry.ControlMode.none)

In [15]:
#cv.destroyAllWindows()

In [16]:
obj = C.addFrame("obj0")

In [11]:
# set frame parameters, associate a shape to the frame, 
obj.setPosition([0.2, .2, 2.])
obj.setQuaternion([1,0,.5,0])
obj.setShape(ry.ST.capsule, [.002,.002])
obj.setColor([0,0,0])
V.setConfiguration(C)
V.recopyMeshes(C) #this is rarely necessary, only when you change meshes within C

In [12]:
#-- the following is the simulation loop
tau = .01

for t in range(300):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q

    V.setConfiguration(C) #to update your model display
    [y,J] = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "obj0"])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y);

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)
    V.setConfiguration(C)

In [14]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
V = ry.ConfigurationViewer()
V.setConfiguration(C)

In [16]:
R_gripper = C.frame("R_gripper")
R_gripper.setContact(1)
obj.setContact(1)

In [17]:
IK = C.komo_IK(False)
IK.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
IK.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter", "obj0"], ry.OT.sos, [1e2]);

In [18]:
# Calling the optimizer (True means random initialization/restart)
IK.optimize()
IK.getReport()

[{'x_dim': 16, 'T': 1, 'k_order': 1, 'tau': 1.0, 'useSwift': False},
 {'order': 1.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [-1, 0],
  'sos_sumOfSqr': 0.15215186186792007},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'QuaternionNorms',
  'vars': [0],
  'eq_sumOfAbs': 0.0},
 {'order': 0.0,
  'type': 'ineq',
  'feature': 'ProxyCost',
  'vars': [0],
  'inEq_sumOfPos': 0.0},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'Default-0-posDiff-R_gripperCenter-WORLD',
  'vars': [0],
  'sos_sumOfSqr': 559.5054817254124}]

In [19]:
C.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(C) #to update your model display

In [20]:
# Move object and reoptimize

# move object
obj.setPosition([.2,.2,1.5])

# copy C into the IK's internal configuration(s)
IK.setConfigurations(C)

# reoptimize
IK.optimize(0.) # 0 indicates: no adding of noise for a random restart
print(IK.getReport())

# grab result
C.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(C) #to update your model display

[{'x_dim': 16, 'T': 1, 'k_order': 1, 'tau': 1.0, 'useSwift': False}, {'order': 1.0, 'type': 'sos', 'feature': 'qItself#32', 'vars': [-1, 0], 'sos_sumOfSqr': 1.0406443480707795e-06}, {'order': 0.0, 'type': 'eq', 'feature': 'QuaternionNorms', 'vars': [0], 'eq_sumOfAbs': 0.0}, {'order': 0.0, 'type': 'ineq', 'feature': 'ProxyCost', 'vars': [0], 'inEq_sumOfPos': 0.0}, {'order': 0.0, 'type': 'sos', 'feature': 'Default-0-posDiff-R_gripperCenter-WORLD', 'vars': [0], 'sos_sumOfSqr': 558.1634747694275}]
